In [ ]:
# pip install -U pymoo

In [ ]:
import numpy as np
import pandas as pd
import json

from data_preprocess import *
from run_ga import *
import matplotlib.pyplot as plt
import pickle

In [ ]:
tet_path = "./data/tet.json"
fdr_path = "./data/all_class_mutation/Chart-1_fdr.json"

with open(tet_path, 'r') as f:
    tet_data = json.load(f)


with open(fdr_path, 'r') as f:
    fdr_data = json.load(f)

chart_1_tet = tet_data['Chart-1']
chart_1_fdr = dict()

for key, value in fdr_data['Chart_1'].items():
    chart_1_fdr[key] = value['mutation-score']


chart_1_tet = sorted(chart_1_tet.items())
chart_1_fdr = sorted(chart_1_fdr.items())

In [ ]:
tests = [i[0] for i in chart_1_tet]
execution_times = np.array([i[1] for i in chart_1_tet])
fault_detections = np.array([i[1] for i in chart_1_fdr])

test_cases = np.column_stack((execution_times, fault_detections))
adequacy_scores = get_adequacy_scores(fault_detections)

In [ ]:
bitflip = run_nsga(test_cases, False)

In [ ]:
adeq = run_nsga_with_adequecy(test_cases, adequacy_scores, False)

In [ ]:
bitflip_res = np.abs(bitflip.F)
adeq_res = np.abs(adeq.F)

plt.scatter(bitflip_res[:, 0], bitflip_res[:, 1], label="Bitflip")
plt.scatter(adeq_res[:, 0], adeq_res[:, 1], label="Adequacy")
plt.xlabel("TET")
plt.ylabel("FDR")
plt.legend()

In [ ]:
with open('data/merged_data/Chart_1.pkl', 'rb') as f:
    data = pickle.load(f)
print(data)